In [1]:
!pip install opendatasets

In [3]:
import opendatasets as od
import pandas as pd

od.download('https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign')

100%|██████████| 612M/612M [00:07<00:00, 84.6MB/s]


In [4]:
import os
import numpy as np
import time
import pandas as pd

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub # pip install tensorflow_hub

In [30]:
batch_size = 60
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "gtsrb-german-traffic-sign/Train-1",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical')

Found 14670 files belonging to 10 classes.
Using 11736 files for training.


In [31]:
validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    "gtsrb-german-traffic-sign/Train-1",
    validation_split=0.2,
    image_size=(img_height, img_width),
    batch_size=10,
      seed=123,
    subset="validation",
    label_mode='categorical'
)

Found 14670 files belonging to 10 classes.
Using 2934 files for validation.


In [32]:
keras_cache_dir = os.path.join(os.getcwd(),'keras_cache')
os.makedirs(keras_cache_dir, exist_ok = True)

# Note: The Keras home directory is hard-coded to
# "~/.keras" (Linux) or "%USERPROFILE%\.keras" on Windows
# so any weights downloaded from "keras.applications" will
# be found there (in the subdirectory "models").

tfhub_cache_dir = os.path.join(os.getcwd(),'tfhub_cache')
os.makedirs(tfhub_cache_dir, exist_ok = True)
os.environ['TFHUB_CACHE_DIR'] = tfhub_cache_dir

In [33]:
class_names = np.array(train_ds.class_names)
print(class_names)

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']


In [34]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

After the import part, we're gonna train our new model.

In [35]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [36]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [37]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 10)                12810     
                                                                 
Total params: 2270794 (8.66 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


the size of the model is 8.66MB


In [38]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [39]:
import keras
tensorboard_logdir = os.path.join(os.getcwd(),'logs','tensorboard') # Let's log Tensorboard data to this folder
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_logdir) # Create a Tensorboard "callback"
history = model.fit(train_ds, epochs=5,validation_data=validation_data,
                    callbacks=[tensorboard_callback,tf.keras.callbacks.EarlyStopping(patience=1)])

Epoch 1/5
196/196 [==============================] - 30s 116ms/step - loss: 1.0819 - acc: 0.6449 - val_loss: 2.6206 - val_acc: 0.1517
Epoch 2/5
196/196 [==============================] - 17s 89ms/step - loss: 0.6026 - acc: 0.8205 - val_loss: 2.7606 - val_acc: 0.1527


My new model works perfectly, we can verify it with th graph of accuracy but also with the response in the images. Know we're gonna save it for the problem 2.

In [40]:
feature_extractor_layer.trainable = True
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['acc'])
model.fit(train_ds, epochs=5,validation_data=validation_data,
                    callbacks=[tensorboard_callback,tf.keras.callbacks.EarlyStopping(patience=0)])

Epoch 1/5
196/196 [==============================] - 80s 277ms/step - loss: 1.2190 - acc: 0.6231 - val_loss: 3.4960 - val_acc: 0.0845
Epoch 2/5
196/196 [==============================] - 52s 266ms/step - loss: 0.5419 - acc: 0.8592 - val_loss: 3.7579 - val_acc: 0.0842


In [17]:
test = tf.keras.preprocessing.image_dataset_from_directory(
  "gtsrb-german-traffic-sign/Test",
  labels=None,
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=1)

Found 12630 files belonging to 1 classes.


In [18]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
test = test.map(lambda x: (normalization_layer(x)))
#AUTOTUNE = tf.data.AUTOTUNE
#test = test.cache().prefetch(buffer_size=AUTOTUNE)

In [41]:
import time
start_time = time.time()
predicted_batch = model.predict(test)
time_prediction = (time.time() - start_time)
print(int(time_prediction))
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

12630/12630 [==============================] - 76s 6ms/step
80


80 seconds to predict

In [20]:
results=pd.read_csv("gtsrb-german-traffic-sign/Test.csv",sep=",")

In [42]:
acc=0
for n in range(len(predicted_label_batch)):
  if int(predicted_label_batch[n].title())==int(results["ClassId"][n]) and int(results["ClassId"][n])<6:
    acc+=1
print(acc,len(results[results["ClassId"].astype(int)<6]["ClassId"]), acc*100/len(results[results["ClassId"]<10]["ClassId"]))

326 3270 6.791666666666667


In [44]:
import flatbuffers
# Convert the model (note: we are converting the model directly from the "model" variable)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()
print(1)
# Let's save the model as "model_2.tflite"
tflite_model2_path = os.path.join(os.getcwd(),'model_2.tflite')
with open(tflite_model2_path, 'wb') as f:
  f.write(tflite_model)


1
